**Parameters (MongoDB and Open-Meteo API)**
- Initializes basic parameters required for MongoDB connection and Open-Meteo API call. 

**Daily Data (Upload Funciton)**
- Defines *get_latest_weather_data* function to access and process most recently available weather measurements (2 days prior to current date), as well as upload it to a MongoDB collection.

In [9]:
import requests
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import pymongo
from pymongo.mongo_client import MongoClient
import certifi
from datetime import datetime, timedelta

# **Parameters (MongoDB and Open-Meteo API)**

### MongoDB Connection

In [10]:
uri = "YOUR MONGO DB URI HERE"

client = MongoClient(uri, tlsCAFile=certifi.where()) # Create client
db = client['Commodity_Weather_Tracker'] # Select database
collection = db['Weather'] # Select collection

### Open-Meteo API Set Up

In [11]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)
url = "https://customer-archive-api.open-meteo.com/v1/archive"

# **Daily Data (Upload Function)**

### *get_latest_weather_data()* Funciton

In [42]:
def get_latest_weather_data():

# ************** REGIONS OF INTEREST **************
# Alter this dict for future region additions/ changes
    city_coordinates = {
        "Des Moines": (41.587874539133864, -93.62025172344258),
        "Harbin": (45.75950681140235, 126.64857767018447),
        "Cuiaba": (-15.596205293870227, -56.09191831845166),
        "Zhengzhou": (34.755190827740414, 113.64387122461706),
        "Ludhianna": (30.901517035526773, 75.85838906910176),
        "Krasnodar": (45.03728005401427, 38.974997420494596),
        "Pergamino": (-33.89135694475014, -60.57281174239956),
        "Changsha": (28.235331625217775, 112.92502052929746),
        "Kolkata": (22.578209269233493, 88.3600522992679),
        "Bangdung": (-6.9167400079749735, 107.61998238624201),
        "Moscow": (55.76611673353855, 37.62160308473536),
        "Saskatoon": (52.158028390626185, -106.66956269182268),
        "Sydney": (-33.86605200565613, 151.195601470287),
        "Belo Horizonte": (-19.91948291035973, -43.943110737706746),
        "Buon Ma Thot": (12.668022378811369, 108.03800228713408),
        "Medellin": (6.247273414666685, -75.56473290099956),
        "San Pedro": (4.758549757562983, -6.643381990584686),
        "Kumasi": (6.667214457444658, -1.6109803431888547),
        "Makassar": (-5.162091928417558, 119.43798624858383),
        "Urumqi": (43.82762776918532, 87.61905395940317),
        "Ahmedabad": (23.029637272051755, 72.58384888080346),
        "Lubbock": (33.57838946952301, -101.85725664222603),
        "Ribiero Preto": (-21.16869931994159, -47.815160933419556),
        "Pune": (18.52492414066214, 73.85678412283815),
        "Bangkok": (13.765109257915768, 100.49778506193263),
        "Orlando": (28.538167830091528, -81.37494078901534),
        "Veracruz": (19.172690753314367, -96.13601049020042)
    }
# ************** REGIONS OF INTEREST **************

# ************** OPEN-METEO API SET UP **************
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)
    url = "https://customer-archive-api.open-meteo.com/v1/archive"
# ************** OPEN-METEO API SET UP **************

    # Date String
    now = datetime.now()
    previous_day = now - timedelta(days=2) # ************** COMPENSATE FOR DATA DELAY **************
    date_string = previous_day.strftime("%Y-%m-%d")

# ************** API PARAMETERS **************
# Alater this dict/ list for any future variable additions/ changes
    common_params = {
        "start_date": date_string,
        "end_date": date_string,
        "daily": ["temperature_2m_mean", "apparent_temperature_mean", "daylight_duration", "sunshine_duration", "precipitation_sum", "rain_sum", "snowfall_sum", "shortwave_radiation_sum", "et0_fao_evapotranspiration"],
        "apikey": "YOUR OPEN-METEO API KEY HERE"
        }
# ************** API PARAMETERS **************

    # Dictionary to hold the DataFrames
    city_dataframes = {}

    # Iterate over the city coordinates
    for city, (lat, lon) in city_coordinates.items():
        params = common_params.copy()
        params.update({"latitude": lat, "longitude": lon})

        # Make the API call
        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]
        
        # Process daily data
        daily = response.Daily()
        daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
        daily_apparent_temperature_mean = daily.Variables(1).ValuesAsNumpy()
        daily_daylight_duration = daily.Variables(2).ValuesAsNumpy()
        daily_sunshine_duration = daily.Variables(3).ValuesAsNumpy()
        daily_precipitation_sum = daily.Variables(4).ValuesAsNumpy()
        daily_rain_sum = daily.Variables(5).ValuesAsNumpy()
        daily_snowfall_sum = daily.Variables(6).ValuesAsNumpy()
        daily_shortwave_radiation_sum = daily.Variables(7).ValuesAsNumpy()
        daily_et0_fao_evapotranspiration = daily.Variables(8).ValuesAsNumpy()

        daily_data = {"date": pd.date_range(
            start=pd.to_datetime(daily.Time(), unit="s", utc=True),
            end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=daily.Interval()),
            inclusive="left"
        )}
        daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
        daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
        daily_data["daylight_duration"] = daily_daylight_duration
        daily_data["sunshine_duration"] = daily_sunshine_duration
        daily_data["precipitation_sum"] = daily_precipitation_sum
        daily_data["rain_sum"] = daily_rain_sum
        daily_data["snowfall_sum"] = daily_snowfall_sum
        daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
        daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration

        # Create the DataFrame and add a column for the city
        df = pd.DataFrame(data=daily_data)
        df['City'] = city
        
        # Save the DataFrame in the dictionary
        city_dataframes[city] = df

    # Convert to MongoDB upload format
    daily_df = pd.concat(city_dataframes.values(), ignore_index=True)
    daily_df.date = pd.to_datetime(daily_df.date).dt.strftime('%Y-%m-%d')
    daily_weather_list = daily_df.to_dict(orient='records')

    # Print statement to confirm execution and provide details
    print(f"Data for {date_string} downloaded successfully on {now.strftime('%Y-%m-%d %H:%M:%S')}")

# ************** MONGODB CONNECTION **************
    uri = "YOUR MONGO DB URI HERE"
    client = MongoClient(uri, tlsCAFile=certifi.where()) # Create client
    db = client['Commodity_Weather_Tracker'] # Select database
    collection = db['Weather'] # Select collection
# ************** MONGODB CONNECTION **************

    # Insert records into MongoDB colleciton
    x = collection.insert_many(daily_weather_list) # Insert record

    # Print statement to confirm execution and provide details
    print(f"Inserted {len(daily_weather_list)} records into the MongoDB collection.")

    

### Be aware there is a 2 day delay in the most recently available weather measurements from Open-Meteo

In [43]:
get_latest_weather_data()

Data for 2024-08-04 downloaded successfully on 2024-08-06 10:41:03


[{'date': '2024-08-04',
  'temperature_2m_mean': 26.970413208007812,
  'apparent_temperature_mean': 30.506502151489258,
  'daylight_duration': 51355.28125,
  'sunshine_duration': 47251.34375,
  'precipitation_sum': 0.0,
  'rain_sum': 0.0,
  'snowfall_sum': 0.0,
  'shortwave_radiation_sum': 24.639999389648438,
  'et0_fao_evapotranspiration': 5.474867343902588,
  'City': 'Des Moines'},
 {'date': '2024-08-04',
  'temperature_2m_mean': 25.67525291442871,
  'apparent_temperature_mean': 28.855283737182617,
  'daylight_duration': 52627.125,
  'sunshine_duration': 18511.796875,
  'precipitation_sum': 11.000000953674316,
  'rain_sum': 11.000000953674316,
  'snowfall_sum': 0.0,
  'shortwave_radiation_sum': 12.920000076293945,
  'et0_fao_evapotranspiration': 2.8079330921173096,
  'City': 'Harbin'},
 {'date': '2024-08-04',
  'temperature_2m_mean': 30.990835189819336,
  'apparent_temperature_mean': 29.122299194335938,
  'daylight_duration': 41284.453125,
  'sunshine_duration': 37450.375,
  'precipi